## SuperMarioBot: Using Deep Q-Learning to Play Super Mario Bros.

### Contributors

Sam Kolins (sk3651)
Atishay Sehgal (as5453)
Arpita Shah (as5451)

### Introduction

In 2013, researchers at Google DeepMind published the now-infamous paper **[Playing Atari with Deep Reinforcement Learning](https://arxiv.org/pdf/1312.5602.pdf)**, in which for the first time, a neural network algorithm learned how to play a video game in a fairly organic sort of way by playing against itself and training on the images produced from those play sessions. Since then, in just five short years, a whole host of enthusiastic data scientists have trained neural networks on a wide variety of different games. Most of these games have been Atari games, but for our project, we wanted to try something a bit more complicated: ***[Super Mario Bros. for the Nintendo Entertainment System (NES)](https://en.wikipedia.org/wiki/Super_Mario_Bros.)***. We are attempting here to train a deep Q-learning algorithm to play *Super Mario Bros.*; in particular, this means we want to get Mario to the end of the level before time runs out and without dying.

### How to Play *Super Mario Bros.*

#### Objective and Level Design

The goal of the game, as mentioned previously, is to get to the end of the level. This is not as easy as it sounds; along the way, Mario must overcome many obstacles in the form of moving enemies that can hurt Mario and bottomless pits that will instantly kill him if he falls into them. Mario also only starts with three lives; if Mario still has lives remaining, he can restart from the beginning of the current level, but **if he runs out of lives, he gets a *Game Over*, and must restart from World 1-1** (although, there is a little-known trick whereby pressing **Start** and **A** to begin a new game will place Mario at the first level of whichever world he died on; so for example, if Mario gets a Game Over on World 4-3, he can restart on World 4-1 instead of World 1-1 using this trick). 

In particular, *Super Mario Bros.* is a side-scrolling platformer, meaning Mario will need to continue moving right in order to access the end of the level. The end of the level is represented by a flag; if Mario touches this flag, the level automatically ends, and a cutscene plays in which Mario enters a castle (if your level timer ends in 1, 3, or 6, a number of fireworks worth 500 points each equal to the last timer digit will fire off). There are eight worlds with four levels apiece, bringing the game's level count to 32; each level is denoted by World $X$-$Y$, where $X$ is the number of the world and $Y$ is the number of the level. There are five kinds of levels (each having their own musical theme in the original NES version of *Super Mario Bros.*):

* **Overworld**: these levels are the typical "default" platforming levels that take place outside atop solid ground. They do not tend to vary much in their terrain. The first level of each world is always an Overworld level; sometimes, the second or even third level of each world is an Overworld level as well.
* **Underground**: at the beginning of each of these levels, Mario travels down a *__Warp Pipe__* (generally used for transportation in the *Mario* games, but can also sometimes contain enemies) where he finds himself underground. These levels have a darker color theme and a ceiling of blocks above Mario at the top of the screen (and more blocks in general). Often times, the second level of each world will be an Underground level.
* **Underwater**: Mario starts the stage, and spends the entire stage, underwater. The physics of the entire level have completely changed as now Mario moves by swimming upward instead of jumping and falls much slower as a result. While Mario is not at any risk of drowning, there are unique swimming enemies in these levels that can still serve as potent threats due to their strange movement patterns (such as [Bloopers](https://i.etsystatic.com/9680903/r/il/26312a/620419040/il_570xN.620419040_rwuo.jpg)). Much like with Underground levels, the second level of each world is often Underwater (if it isn't Underground).
* **Athletic**: These levels typically have little to no true ground areas at all as Mario traverses on the tops of [large trees](https://www.mariowiki.com/images/d/dd/SMB1W1-3.png). They may also make use of moving platforms (see previous pic). This increased verticality and general increase in bottomless pits can make them treacherous to navigate. The third level in each world is almost always Athletic with the exception of World 8-3, the game's penultimate level, which takes place in the Overworld.
* **Castle**: In the last level of each world (so World $X$-$4$), Mario enters one of Bowser's castles (Bowser being the main villain of *Super Mario Bros.* and much of the *Mario* series in general). Much like the Underground levels, these levels are generally enclosed by a ceiling because they canonically take place indoors. Generally speaking, the most treacherous of obstacles will appear in these levels, many of which having a fire theme; bottomless pits are generally replaced by [lava pits](https://www.mariowiki.com/images/3/32/SMB1W1-4.png) (which can spit out their own fireballs called [Podoboos](https://vignette.wikia.nocookie.net/mario/images/4/48/Podoboo_1.jpg/revision/latest?cb=20120504221020) with little to no warning) and [Fire-Bars](https://www.mariowiki.com/images/3/3c/SMBW5-4.png) which can [spin around](https://www.mariowiki.com/images/7/74/FireRodSMB.gif) and hit the player are numerous. Bowser, standing on a bridge overlooking a lava pit, is directly blocking the end of the level (represented by an axe instead of a flag) and will spit fire traveling in straight lines at the player when he appears on-screen. If Mario touches this axe, the bridge disappears, Bowser falls into the pit, and Mario clears the level and the world, shortly followed by an [infamous cutscene](https://www.youtube.com/watch?v=n4ucO4xe28c).

After beating the NES version of the game, the player unlocks a **hard mode**. This makes many modifications to the game, like turning [Goombas](https://ctl.s6img.com/society6/img/vG1nc6QSu4HFMq-cU71ibAx76z4/w_700/prints/~artwork/s6-0020/a/7729279_15115587/~~/pixel-goomba-super-mario-bros-prints.jpg), the game's weakest and most standard enemies, into [Buzzy Beetles](http://farm1.static.flickr.com/59/194964545_e89f8ff3e7.jpg) which are immune to fireballs. All enemies move faster, power-ups are no longer available, all moving platformers are made smaller, and Fire-Bars are far more common. Levels in the base game that are similar to each other (e.g. World 1-3 and 5-3 are almost the same except 5-3 is harder) are now made identical, making a lot of easier levels harder.

#### Controls

The [NES controller](https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5579/5579396_sd.jpg;maxHeight=640;maxWidth=550) has eight buttons: the ***D-pad*** (containing the four cardinal directions **up**, **down**, **left**, and **right**), **A**, **B**, **Start**, and **Select**. The function of each button is generally intuitive, but depends on context:

* **Left** and **right** will move Mario left and right. The other two D-pad directions, **up** and **down**, are rarely used, but do have uses; **down** is used to access Warp Pipes and crouch as ***Super Mario*** (more on that later) while **up** is generally only used for traveling up vines.
* **A** is used to jump or swim upwards.
* **B** is used to run, throw a ***fireball*** (more on this later), restart the game at the end, or select a world.
* **Start** pauses the game and confirms the selected option on the title screen.
* **Select** is only used on the title screen to select the number of players; *Super Mario Bros.* was a multiplayer game on the NES where a second player could play as Mario's brother, Luigi. Control would alternate between the two players whenever any of them died.

#### Items and Enemies

As mentioned before, each level has enemies that can hurt Mario if he touches them. Typically, the only way Mario can beat enemies is by jumping on them (although there are exceptions, like [Spinies](https://c1.staticflickr.com/1/67/194964203_92e02e0e72_m.jpg), which have spikes on their back that will hurt Mario if he touches them). Mario typically cannot survive a hit; that is, if an enemy touches him once (not counting Mario jumping on them), he dies and loses a life. However, Mario can collect an item known as a **[Super Mushroom](http://icons.iconarchive.com/icons/ph03nyx/super-mario/256/Retro-Mushroom-Super-icon.png)** (technically called a "Magic Mushroom" in the days of *Super Mario Bros.* and has since changed names) that will make him grow to twice his size. This allows Mario to break blocks above him (except the [? Block](https://res.cloudinary.com/teepublic/image/private/s--xm7dVDxa--/t_Preview/b_rgb:0195c3,c_limit,f_jpg,h_630,q_90,w_630/v1481855557/production/designs/955552_1.jpg), from which Mario can collect [coins](https://i.kinja-img.com/gawker-media/image/upload/s--dpkSgaoo--/c_fit,f_auto,fl_progressive,q_80,w_320/18j2qh36rzq56jpg.jpg) and items), crouch, and more importantly, take a hit without dying (although doing so will transform him back to his regular size). Mario can also collect a **[Fire Flower](http://icons.iconarchive.com/icons/ph03nyx/super-mario/256/Retro-Flower-Fire-icon.png)** which, in addition to granting Mario the effects of the Super Mushroom, also gives him the part to throw fireballs with **B** that can kill enemies (even Spinies). In this state, Mario has a distinct look - **Fire Mario** - in which his shirt turns red and his hat and overalls turn white. A comparison between all three standard versions of Mario can be found [here](https://i.imgur.com/q7FzrbE.png). Lastly, Mario can very rarely find a [Starman](http://rs997.pbsrc.com/albums/af99/Super_Mario_11/SMB1StarMan.gif~c200) inside a ? Block that grants him invincibility to any enemy he touches and also changes the music to this [catchy theme](https://www.youtube.com/watch?v=Lw049q22-hY) during the duration of its effect.

### OpenAI Gym and `gym-super-mario-bros`

The version of the game we are using for this project is not exactly the same as the NES version of Mario, however, as we do not have access to an authentic NES console. Instead, we are **using an NES emulator to run the game**, modified to run in Python (2.7, 3.5, or 3.6) and with some extra modifications for convenience. The package we are using, called **[`gym-super-mario-bros`](https://pypi.org/project/gym-super-mario-bros/)**, was created by Christian Kauten ("kautenja" on GitHub) and is an OpenAI Gym environment using the [`nes-py` emulator](https://pypi.org/project/gym-super-mario-bros/) (also made by Kauten) that can run both the original *Super Mario Bros.* and *Super Mario Bros. 2: The Lost Levels*. We are only concerned with *Super Mario Bros.* for this project.

There are many changes made to the game, like running at a far smaller resolution than the NES was capable of and the lack of music, but the most important changes are summarized here:

- Nearly all __cutscenes__ have been __removed from the game__. This includes Mario's death animation, the cutscenes that display the current level name and Mario's life count prior to the beginning of each level, the level clear cutscene (where Mario enters the castle just beyond the flag), the world clear cutscene, and so on. This is because the game cannot register inputs during these scenes and so there is no reason to have the neural network attempt to train on them. If a cutscene can't be removed by hacking the NES's RAM (as the environment attempts to do from `nes-py`), the program will lock any Python process attempting to train on the cutscene frames until the game is ready to accept inputs again.
- __Levels can be loaded individually.__ Rather than play from the very beginning of the first level, we can instead decide to train a network on any particular level of our choosing. This can be a good way to train Mario on trickier levels that may have more difficult terrain (like Athletic or Castle levels) or enemies that only appear in certain levels. Doing so however means that a single episode contains only one life rather than three lives.
- __There are multiple downsampled versions of the game.__ Downsampling is important because the reduction in rendering detail will make it easier for the convolutional layers of the deep Q-learning network to process the game images. There are three downsampled versions, in addition to `v0`, the original version of the game (see the `gym-super-mario-bros` documentation for more details):
    * `v1` does not affect any foreground elements, but removes color from the background and simplifies the designs of background elements somewhat.
    * `v2` further simplifies all in-game assets, including Mario and the in-game heads-up display (HUD), into blockier designs.
    * `v3` takes this even further, simplifying all elements into colored rectangles (including Mario himself!).
- The base game contains **4 frames of frameskip**. This means that, though each frame is calculated, only every fourth frame is drawn when the game is rendered. **This frameskip can be removed**, but it is not recommended for CPU's/GPU's that are not fast enough to render the game quickly.

There are many more details about how the game tracks various statistics that we would like to optimize over (like Mario's x-position) or against (like the in-game clock). Please read the `gym-super-mario-bros` documentation for more details.

### Final Goal

There are many possible goals we can set for ourselves here. A very ambitious goal would be to train Mario to beat the entire game in a single episode... all 32 levels! We're going to be more realistic though, especially given our relative inexperience in Python and lack of time relative to the length of training times and say that a good goal would simply be to **beat World 1-1**. If not, we'd like to at least see how far we can get and offer possible improvements or reasons as to why we were unable to reach this goal.

### Additional Notes

This entire notebook was created *locally* in Python 3.6.6 because ***Colab has issues running `gym-super-mario-bros`*** (at least, for us; trying to load the Mario environment was giving us weird errors we were unable to diagnose). There it is ***strongly* recommended that you *do not attempt to run this in Google Colab***. Also, make sure that **Microsoft Visual C++ 14.0 is installed**, which can be acquired [here](https://www.scivision.co/python-windows-visual-c++-14-required/).

And now, without further ado, let the learning begin!

In [ ]:
# note: this entire notebook was created LOCALLY in Python 3.6.6 because Colab has issues running gym-super-mario-bros

# for me:
# to access, open Anaconda Prompt and type "activate py36" followed by "jupyter notebook"
# it might only need to be done once
# does not need to be done on the desktop because I made sure to install Anaconda3 5.2.0 which has Python 3.6 by default

# in console: "python -m pip install --upgrade pip" after "activate py36" to enter the Python 3.6 notebook
# start by installing the package
!pip install gym-super-mario-bros

When running the starter code, I had some missing DLL errors from trying to import `cv2` (which is a dependency used by either `nes_py` or `gym_super_mario_bros`), so installing the following package might help if you get the same issues. If not, I would recommend installing [Dependency Walker](http://dependencywalker.com/), which is free, which I ran on the `.pyd` file for my version of `cv2` to determine which dependencies I was missing and thereby manually downloaded them.

In [6]:
# I had some missing DLL errors from trying to import cv2, so installing this might help if you have the same issues
# if not, you might have to look for the missing DLL's manually. there's a free program called Dependency Walker that can help
# locate missing DLL's
!pip install opencv-contrib-python

We mentioned earlier that the NES controller has eight buttons. Because any combination of inputs can be registered on a single frame, that means there are $2^8 = 256$ possible NES controller inputs possible on a given frame. But *Super Mario Bros.* fails to make use of most of them; as discussed earlier, the **Select** and **Start** buttons (and even **up** and **down**) are highly niche buttons within the game, and because this emulation lacks a title screen (as there is no reason to train on it), we never need to press **Start** or **Select**. This seems like it would an extremely wasteful use of this action space, but luckily, `gym_super_mario_bros` contains three action libraries. One of these is `SIMPLE_MOVEMENT`, which contains the following seven inputs: **idle**, **right**, **right+A**, **right+B**, **right+A+B**, **A**, and **left**. `COMPLEX_MOVEMENT` adds button combinations that involve the direction **left** while `RIGHT_ONLY` removes any non-idle action that does not move Mario to the right.

Why might Mario ever want to move left? He may want to dodge enemies or go back to get missed coins or power-ups, but there is a more pertinent use case here. Mario may be forced to enter a pipe that opens sideways, in which case if it is possible to jump on that pipe and keep moving forward past the opening like it is at [the end of World 1-2](https://www.mariowiki.com/images/f/fb/World_1-2_SMB.png), Mario is very likely to get stuck running into the wall unless he learns to move left.

In [12]:
# initializing an environment in which Mario chooses a random action from the SIMPLE_MOVEMENT action library for 5000 steps
# on the default Mario ROM's, there is a frameskip factor of 4 frames, so one step is really more like 4 frames

from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v2')
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

Now, we assemble the network architecture as an object instance of the class `CNN_DQNAgent`. The first half is a collection of five convolutional network layers; the second half is, for lack of a better term, going to be called the ***AV-stream***. This splits the output of the final convolutional layer into a *value stream* and an *advantage stream*. The former represents how well off Mario is in his current state; the latter represents how much better off Mario can be by taking a particular action. Advantage is essentially the difference between the Q function and the value stream. To get the estimated Q output, we must add back the advantage and value streams together; the highest advantage would then, of course, yield the highest Q output and therefore represent our optimal action for the current state.

The basis of this network design was drawn from [Branko Blagojevic's great tutorial](https://medium.com/ml-everything/learning-from-pixels-and-deep-q-networks-with-keras-20c5f3a78a0) on reinforcement learning with CNN's and Keras.

In [9]:
# we can start to actually implement the deep Q-network (DQN) now
# if you get a FutureWarning message about "np.floating" being deprecated or something, ignore it

from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, Input, Lambda
import keras.backend as K

class CNN_DQNAgent():
    def __init__(self, input_shape, num_actions, final_layer_size = 128):
        # The input image of Super Mario Bros. is 256x240x3 (RGB color channels)
        self.inputs = Input(shape = [input_shape], name = "main_input")
        
        # 5 convolutional layers (with relu activation) on the input
        # Conv1: 32 filters, 24x24 with 4x4 striding
        self.model = Conv2D(filters = 32, kernel_size = [8, 8], strides = [4, 4], 
                            activation = "relu", padding = "valid", name = "conv1")(self.inputs)
        # Conv2: 64 filters, 17x13 with 2x2 striding
        # the rectangular filters here are designed to make the output square (not counting depth of course)
        self.model = Conv2D(filters = 64, kernel_size = [17, 13], strides = [2, 2], 
                            activation = "relu", padding = "valid", name = "conv2")(self.model)
        # Conv3: 64 filters, 12x12 no striding (1x1)
        self.model = Conv2D(filters = 64, kernel_size = [12, 12], strides = [1, 1], 
                            activation = "relu", padding = "valid", name = "conv3")(self.model)
        # Conv4: 128 filters, 8x8 no striding
        self.model = Conv2D(filters = 128, kernel_size = [8, 8], strides = [1, 1], 
                            activation = "relu", padding = "valid", name = "conv4")(self.model)
        # Conv5: 128 (final_layer_size) filters, 4x4 no striding
        self.model = Conv2D(filters = final_layer_size, kernel_size = [4, 4], strides = [1, 1], 
                            activation = "relu", padding = "valid", name = "conv5")(self.model)
        # output size is now 1x1x128 with 1,884,576 trainable parameters
        
        # the final conv layer is separated into value and advantage streams
        # value stream: current value in a given state
        # advantage stream: change in value after making a particular move
        # Advantage(state, action) = Q(state, action) - Value(state)
        self.stream_AC = Lambda(lambda layer: layer[:, :, :, :final_layer_size // 2], 
                                name = "adv_stream")(self.model)
        self.stream_VC = Lambda(lambda layer: layer[:, :, :, final_layer_size // 2:], 
                                name = "val_stream")(self.model)
        
        # flatten the advantage and value functions
        self.stream_AC = Flatten(name = "adv_flat")(self.stream_AC)
        self.stream_VC = Flatten(name = "val_flat")(self.stream_VC)
        
        # define weights for advantage and value layers
        # these will be trained so that the matmul matches the expected adv/val from play
        self.Advantage = Dense(num_actions, name = "final_advantage")(self.stream_AC)
        self.Value = Dense(1, name = "final_value")(self.stream_VC)
        
        # add value and advantage to get Q-output
        # advantage is evaluated based on how good the action is relative to average state adv
        self.model = Lambda(lambda val_adv: val_adv[0] + (val_adv[1] - K.mean(val.adv[1], axis = 1, keepdims = True)),
                           name = "final_out")([self.Value, self.Advantage])
        self.model = Model(self.inputs, self.model)
        self.model.compile("adam", "mse")
        self.model.optimizer.lr = 0.0001 # learning rate

Now, we wish to implement an ***experience replay system*** that stores game experiences. This too is represented as an instance of a class, this time called `ExperienceReplay`. To train from the network, we sample a batch of these from the trove of game experiences we have collected. We also cycle out older experiences for newer, more relevant ones past a certain limit defined here by `buffer_size`. Again, Blagojevic's tutorial was very helpful here.

In [10]:
import numpy as np

class ExperienceReplay:
    def __init__(self, buffer_size = 50000):
        """ Data structure used to hold experiences """
        # each element of buffer contains a list: [state, action, reward, next_state, done]
        self.buffer = []
        self.buffer_size = buffer_size
        
    def add(self, experience):
        """ Adds list of experiences to the buffer """
        # extend stored experiences
        self.buffer.extend(experience)
        # keep last buffer_size number of experiences
        self.buffer = self.buffer[-self.buffer_size:]
        # this construction better than using a control statement on current length of buffer
        
    def sample(self, size):
        """ Returns a sample of experiences from the buffer """
        sample_idxs = np.random.randint(len(self.buffer), size = size)
        sample_output = [self.buffer[idx] for idx in sample_idxs]
        sample_output = np.reshape(sample_output, (size, -1))
        return sample_output

As mentioned, we occasionally pull from `ExperienceReplay` to train the network. We then continue playing using the updated network after the weights have been changed. The **target network** is used to determine what the values should be based on the current state and action. In the next section, we will actually implement how training the main model is supposed to work, but first, we must initialize a host of useful parameters that will be used for training. Each line is commented with a brief description of what each variable represents.

In [13]:
# initializing useful parameters
batch_size = 64 # number of experiences used for each training step
num_epochs = 10 # number of epochs to train
update_freq = 1 # frequency of network updates
y = 0.99 # discount factor
prob_random_start = 1 # initial epsilon (chance of random action in epsilon-greedy policy)
prob_random_end = 0.1 # final epsilon
epsilon_steps = 333 # number of times epsilon decays from init -> final (should be approx. num_episodes)
num_episodes = 333 # number of episodes to train; remember that one episode = 3 lives!
pre_train_episodes = 25 # number of episodes of random action (to build replay buffer)
max_num_step = 50000 # maximum episode length in units of steps
load_model = True # if True, will load a saved model
path = "./dqn" # path for saved model
final_layer_size = 128 # size of final conv layer (before AV-stream split)
tau = 0.001 # rate to update primary network toward target network

Some useful notes:

* Remember that a single episode consists of three lives. Since we aren't accounting for changes in behavior based on the number of lives as that is not a part of the reward function (to be discussed later), our strategy is independent of the current life count, and so one-life episodes would be indistinguishable from three-life episodes. We haven't adjusted the *Super Mario Bros.* package to reflect this difference (although we could choose to load a single level if we wanted one-life episodes), but it does mean we can run on fewer episodes and still get good results. Choosing $333$ episodes, for example, as I have done here by default, still gets us $333 \times 3 = 999$ lives worth of training.
* This algorithm uses an $\epsilon$-greedy policy, but introduces an interesting variation in which $\epsilon$ will vary over time. Mario will therefore be highly experimental in the earliest stages of the stage, trying a host of different options to collect as many game states as possible (to say nothing of the pre-training episodes in which Mario will only act randomly), but will be increasingly looking to optimize his reward over time.
* We want `max_num_step` to be very large. As you might have seen running a completely random sample of Mario actions a few code blocks ago, $5000$ steps is enough for an episode or two, but that's about it. Once Mario gets better at running through levels, each episode could be very long; we don't want to cut off too many potentially successful level completion runs early.
* The inclusion of `final_layer_size` is technically unnecessary as we have given it the default value of `128` when we were defining our network agent. Because it is important in determining the sizes of our advantage and value streams though, it feels like something that should technically be modifiable via changing a variable.